# scVelo on HSCs
## Preprocessing: denoising + batch correction

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%matplotlib inline
import copy
import numpy as np
import pandas as pd
import scvelo as scv
import scanpy.api as sc
import matplotlib.pyplot as plt
import anndata as ad
import scipy
import seaborn as sns
sc.set_figure_params(scanpy=True, dpi_save=200)

# Load, clean, annotate Loom files

In [4]:
adata = scv.read('velocyto/sce_hspc.loom')
bdata = scv.read('velocyto/nov28_hspc.loom')

anno1 = pd.read_csv("hsc_anno1.csv")
anno2 = pd.read_csv("hsc_anno2.csv")

KeyboardInterrupt: 

In [ ]:
def pre_processing(data, anno):
    # Make variable names unique
    data.var_names_make_unique()
    # clean up the data
    scv.pp.cleanup(data, clean='all')
    
    # format anno
    anno.index = anno['cell']
    anno = anno.drop(columns='cell')
    
    # Keep cells that passed QC
    cells_to_keep = anno.index.values
    data = data[cells_to_keep]
    
    # Add metadata 
    data.obs['Donor'] = anno['Donor']
    data.obs['Plate'] = anno['Plate']
    data.obs['CD45RA'] = anno['CD45RA']
    data.obs['CD34'] = anno['CD34']
    data.obs['CD38'] = anno['CD38']
    data.obs['CD49f'] = anno['CD49f']
    data.obs['CD90'] = anno['CD90']
    data.obs['log10_total_counts'] = anno['log10_total_counts']
    data.obs['log10_total_features_by_counts'] = anno['log10_total_features_by_counts']
    data.obs['pct_counts_MT'] = anno['pct_counts_MT']
    data.obs['S.Score'] = anno['S.Score']
    data.obs['G2M.Score'] = anno['G2M.Score']
    data.obs['Cycle.Score'] = anno['Cycle.Score']
    data.obs['Cycling'] = anno['Cycling']
    data.obs['Stemness.Score'] = anno['Stemness.Score']
    data.obs['DNMT3A.Score'] = anno['DNMT3A.Score']
    data.obs['DNMT3A_mut'] = anno['DNMT3A_mut']
    data.obs['median_gene_length'] = anno['median_gene_length']
    
    return data

In [ ]:
# do pre-processing
adata = pre_processing(adata, anno1)
bdata = pre_processing(bdata, anno2)

In [ ]:
# splice and unspliced matrices
spliced = scipy.sparse.vstack([adata.layers['spliced'], bdata.layers['spliced']])
unspliced = scipy.sparse.vstack([adata.layers['unspliced'], bdata.layers['unspliced']])

spliced

In [ ]:
# Concatenate data
adata = adata.concatenate(bdata)
adata.layers['spliced'] = spliced
adata.layers['unspliced'] = unspliced

adata

## Denoising and Batch Correction

In [ ]:
%%time
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.dca(adata)

In [27]:
## remove outlier
adata = adata[adata[:, ['CDK6']].X < max(adata[:, ['CDK6']].X), :]

In [28]:
ddata = copy.deepcopy(adata)

In [29]:
adata = copy.deepcopy(ddata)

In [92]:
%%time

### BATCH CORRECTION
scv.pp.filter_and_normalize(adata, min_counts=20, min_counts_u=10, n_top_genes=10000)
#scv.pp.filter_and_normalize(adata, min_counts=20, min_counts_u=10, n_top_genes=6000)

#sc.pp.regress_out(adata, ['pct_counts_MT'])
        ### See what is feasible to regress out and what is not

bdata = copy.deepcopy(adata)

# split into 4 donors
bm1 = adata[adata.obs.Donor.str.contains("BM1")]
bm2 = adata[adata.obs.Donor.str.contains("BM2")]
bm3 = adata[adata.obs.Donor.str.contains("BM3")]
bm4 = adata[adata.obs.Donor.str.contains("BM4")]

# mnn correct
corrected = sc.pp.mnn_correct(bm4, bm3, bm2, bm1, batch_categories= ["BM4", "BM3", "BM2", "BM1"])
adata = corrected[0]

# fix layers
adata.layers['spliced'] = bdata.layers['spliced']
adata.layers['unspliced'] = bdata.layers['unspliced']

Filtered out 41783 genes that are detected in less than 20 counts (spliced).
Filtered out 4968 genes that are detected in less than 10 counts (unspliced).
Normalized count data: X, spliced, unspliced.
Logarithmized X.
Performing cosine normalization...
Starting MNN correct iteration. Reference batch: 0
Step 1 of 3: processing batch 1
  Looking for MNNs...
  Computing correction vectors...
  Adjusting variance...
  Applying correction...
Step 2 of 3: processing batch 2
  Looking for MNNs...
  Computing correction vectors...
  Adjusting variance...
  Applying correction...
Step 3 of 3: processing batch 3
  Looking for MNNs...
  Computing correction vectors...
  Adjusting variance...
  Applying correction...
MNN correction complete. Gathering output...
Packing AnnData object...
Done.
CPU times: user 1min 41s, sys: 19.1 s, total: 2min
Wall time: 38.2 s


In [ ]:
adata.write_h5ad("scRNA_imputed2.h5ad")